In [1]:
import base64
import json
import boto3

ENDPOINT_NAME = "sapiens-segmentation-endpoint"
IMAGE_PATH = "man.png"
OUT_MASK_PATH = "mask.png"

# If your endpoint is in a specific region, set it here (optional).
# region_name = "ap-south-1"
# smrt = boto3.client("sagemaker-runtime", region_name=region_name)
smrt = boto3.client("sagemaker-runtime")

# 1) Read input image and encode as base64 (matches your input_fn JSON path)
with open(IMAGE_PATH, "rb") as f:
    img_bytes = f.read()

payload = {"image_b64": base64.b64encode(img_bytes).decode("utf-8")}
body = json.dumps(payload).encode("utf-8")

# 2) Invoke endpoint
resp = smrt.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Accept="application/json",
    Body=body,
)

# 3) Parse response JSON
resp_body = resp["Body"].read()
result = json.loads(resp_body.decode("utf-8"))

# 4) Decode and save returned mask PNG
mask_png_b64 = result["mask_png_b64"]
mask_png_bytes = base64.b64decode(mask_png_b64)

with open(OUT_MASK_PATH, "wb") as f:
    f.write(mask_png_bytes)

print("Saved mask to:", OUT_MASK_PATH)
print("Mask shape (H, W):", result.get("mask_shape_hw"))


Saved mask to: mask.png
Mask shape (H, W): [627, 413]
